<a href="https://colab.research.google.com/github/robocarstore/donkey-car-training-on-google-colab/blob/master/Donkey_Car_Training_using_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Donkey Car Training using Google Colab

Train your donkey car model using accelerated GPU for FREE on Google Colab

## Check GPU and Tensorflow version
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.

In [ ]:
!nvidia-smi

Verified tensorflow version: > 2.9 and <= 2.15

In [ ]:
import tensorflow
print(tensorflow.__version__)

## Git Clone the donkey repository

In [3]:
!git clone https://github.com/robocarstore/donkeycar.git
%cd /content/donkeycar

Cloning into 'donkeycar'...
remote: Enumerating objects: 15444, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 15444 (delta 17), reused 22 (delta 11), pack-reused 15396
Receiving objects: 100% (15444/15444), 99.88 MiB | 32.31 MiB/s, done.
Resolving deltas: 100% (9907/9907), done.
/content/donkeycar


In [4]:
!git checkout v5

Branch 'v5' set up to track remote branch 'v5' from 'origin'.
Switched to a new branch 'v5'


## Install donkey car

In [5]:
# !pip3 install --upgrade pip setuptools wheel
!pip3 install -e .[pc]

Obtaining file:///content/donkeycar
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.2/21.2 MB 60.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=568819e4dcbb5bd49ac6625da616daff79e4698236a59658a2702b6fbc033f4a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9612 sha256=8c99f89e3bfb59894515df7983722b40b4725ad9c40e3ffbd066de06fbd3927f
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a416

## Create Project

In [6]:
!donkey createcar --path /content/mycar

________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v5.0.dev3 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: complete
Copying car config defaults. Adjust these before starting your car.
Copying train script. Adjust these before starting your car.
Copying calibrate script. Adjust these before starting your car.
Copying my car config overrides
Donkey setup complete.


### Download your dataset

You can download your dataset to this Colab instance by the following command. You might obtain the uuid of the dataset from the meta.json under each tub.

In [ ]:
%cd /content/mycar/data
!curl  -O -J "https://robocarstore-dc-sg.s3.amazonaws.com/tubs/63783f2f-9497-460b-a6ad-3607c20dcbff/data.tar.gz"


Use the following command to extract the dataset into a folder named tub_xxxxxxx

In [ ]:
!tar -xzf data.tar.gz

Alternatively, use the "Getting UUID" notebook to generate the list of tubs uuid for training

In [8]:
tub_uuids = ['b7da4c7a-d868-46e1-a2e7-1b02c472c8ef', 'a38a0958-baba-4b65-aded-e60e2f22e265']

In [9]:
for tub_uuid in tub_uuids:
  !curl -J "https://robocarstore-dc-sg.s3.amazonaws.com/tubs/{tub_uuid}/data.tar.gz" -o /tmp/data.tar.gz
  !tar -zxf /tmp/data.tar.gz -C /content/mycar/data


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.9M  100 21.9M    0     0  7630k      0  0:00:02  0:00:02 --:--:-- 7628k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26.3M  100 26.3M    0     0  8258k      0  0:00:03  0:00:03 --:--:-- 8259k


In [ ]:
!ls

Assuming the latest created folder is the tub_name.

In [11]:
%cd /content/mycar/data

tub_name=!ls -td -- */ | head -n 1
tub_name = tub_name[0].strip('/')
tub_name

/content/mycar/data


'tub_16_23-11-30'

Check whether the data is there

In [ ]:
!ls {tub_name}

Check number of images in this tub

In [ ]:
%cd /content/mycar/data/{tub_name}
!find . -type f -name "*.jpg" | wc -l

/content/mycar/data/tub_49_24-01-13
6623


### Generate a movie for preview

In [12]:
!pip uninstall keras-vis
!pip install git+https://github.com/sctse999/keras-vis



  Cloning https://github.com/sctse999/keras-vis to /tmp/pip-req-build-1v0t_6g8
  Running command git clone --filter=blob:none --quiet https://github.com/sctse999/keras-vis /tmp/pip-req-build-1v0t_6g8
  Resolved https://github.com/sctse999/keras-vis to commit 93e1e803dc0779e25f95b6c1e871d499414e5404
  Preparing metadata (setup.py) ... done
  Created wheel for keras-vis: filename=keras_vis-0.5.0-py2.py3-none-any.whl size=30956 sha256=3c64613095e69adf0c30aa2fda21f1385f4f9a32108eff5de141867d854153b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-z68ih_1r/wheels/76/f4/c7/79f07c4e592d76b0adc5dc32b610988e36a56e8d288d286612
Successfully built keras-vis


In [14]:
%cd /content/mycar
!donkey makemovie --tub ./data/tub* --out {tub_name}.mp4 --start 1 --end 2000

/content/mycar
________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v5.0.dev3 ...
usage: makemovie [-h] [--tub TUB] [--out OUT] [--config CONFIG] [--model MODEL] [--type TYPE]
                 [--salient] [--start START] [--end END] [--scale SCALE] [--draw-user-input]
makemovie: error: unrecognized arguments: ./data/tub_16_23-11-30


Download this movie

In [ ]:
from google.colab import files

files.download(f"{tub_name}.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Train your model

In [ ]:
%cd /content/mycar

model_name = "model1"

!donkey train --tub ./data/tub* --model models/{model_name}

/content/mycar
________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v5.0.dev3 ...
INFO:donkeycar.config:loading config file: ./config.py
INFO:donkeycar.config:loading personal config over-rides from ./myconfig.py
2024-02-19 08:19:55.468604: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 08:19:55.468665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already b

### Quiz
1. How many records are used for training?
2. How many records are used for validation?
3. Does the sum of this number adds up to the number of images in the dataset?
4. What is the final lost value?
5. How long does this training last?
6. How many epoches in this training?
7. How many steps in this epoch?
8. What's the value of `MAX_EPOCHS` in `myconfig.py`?
9. Why the training stop before it reaches the 100th epoch?

Check if the model is generated



In [ ]:
!ls -alh /content/mycar/models/{model_name}

In [ ]:
%cd /content/mycar/models/{model_name}

import glob
file = glob.glob("*.png")

from IPython.display import Image
Image(file[0])

## Copy the trained model back to Donkey Car (Pi)

Once the training is complete on colab, download the model file under /content/mycar/models/ folder location.

In [ ]:
from google.colab import files

files.download('./mypilot.h5')

Alternatively, you can copy the model back to Google Drive too

In [ ]:
!cp /content/mycar/models/mypilot.h5 /content/drive/My\ Drive/mycar/models/mypilot.h5

### Copy the file from your PC or Mac to the Raspberry Pi using Filezilla or scp command.

```
sftp pi@raspberry.local
cd mycar/models
put mypilot.h5
```

## Start Autopilot on Pi


```bash
cd ~/mycar
python manage.py drive --model models/mypilot.h5 --js
```


## Changing training parameters

### Change number of epoch

1. Edit `myconfig.py`
2. Change `MAX_EPOCHS` to 5, 10 , 20
3. Training the model again by executing the training script above
4. What is `MAX_EPOCHS` used for?
5. Why do we want to change `MAX_EPOCHS`?

### Change TRAIN_FILTER

Change TRAIN_FILTER in `myconfig.py` to > 0.1 and check the number of images used for training and validation

### Change Batch Size

1. Edit `myconfig.py`
2. Change `BATCH_SIZE` to 256
3. Training the model again by executing the training script above
4. Does the training go faster? Why?


## Bonus - Salient Object Visualization

In [ ]:
!pip install imageio-ffmpeg


In [ ]:
%cd /content/mycar
!donkey makemovie --tub data/{tub_name} --model models/mypilot.h5 --type linear --salient

In [ ]:
from google.colab import files

%cd /content/mycar
!ls -ahl
files.download('tub_movie.mp4')